<a href="https://colab.research.google.com/github/kangkanM/my_kitchen/blob/master/topic%20modelling%20using%20LDA(gensim)%20on%20news%20data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Topic modeling using LDA(gensim)#

data source = https://www.kaggle.com/therohk/million-headlines/data

In [70]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

loading data


In [21]:
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
documents = pd.DataFrame(data.iloc[:,1])
documents["index"] = documents.index
documents.head()

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


utility fundtions

In [0]:
def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text, min_len= 3):
        if token not in gensim.parsing.preprocessing.STOPWORDS:
            result.append(lemmatize(token))
    return result

In [0]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

In [0]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

In [0]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [0]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [50]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 175 ("govt") appears 1 time.
Word 260 ("group") appears 1 time.
Word 314 ("vote") appears 1 time.
Word 644 ("local") appears 1 time.
Word 923 ("want") appears 1 time.
Word 4091 ("compulsory") appears 1 time.
Word 4092 ("ratepayers") appears 1 time.


In [0]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [57]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.023*"government" + 0.013*"new" + 0.013*"school" + 0.012*"say" + 0.012*"abc" + 0.011*"cut" + 0.010*"job" + 0.009*"minister" + 0.008*"morrison" + 0.008*"turnbull"
Topic: 1 
Words: 0.016*"miss" + 0.010*"flood" + 0.010*"age" + 0.010*"search" + 0.010*"nrl" + 0.009*"victorian" + 0.009*"hobart" + 0.008*"ban" + 0.008*"afl" + 0.008*"sydney"
Topic: 2 
Words: 0.025*"election" + 0.022*"queensland" + 0.013*"tasmania" + 0.012*"nsw" + 0.012*"speak" + 0.012*"work" + 0.011*"country" + 0.011*"power" + 0.010*"end" + 0.010*"drum"
Topic: 3 
Words: 0.024*"win" + 0.020*"market" + 0.018*"die" + 0.017*"world" + 0.014*"cup" + 0.012*"bank" + 0.011*"take" + 0.009*"train" + 0.009*"beat" + 0.008*"scott"
Topic: 4 
Words: 0.043*"australia" + 0.020*"day" + 0.018*"south" + 0.017*"china" + 0.016*"north" + 0.015*"test" + 0.010*"west" + 0.009*"war" + 0.008*"australias" + 0.008*"rescue"
Topic: 5 
Words: 0.018*"health" + 0.016*"federal" + 0.012*"gold" + 0.012*"help" + 0.012*"coast" + 0.009*"public" + 0.00

###Running LDA with TF-IDF###

In [0]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

In [58]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.009*"australia" + 0.009*"world" + 0.008*"cup" + 0.008*"win" + 0.006*"day" + 0.005*"korea" + 0.005*"test" + 0.005*"australian" + 0.005*"cricket" + 0.005*"ash"
Topic: 1 Word: 0.024*"trump" + 0.009*"abc" + 0.008*"climate" + 0.007*"abuse" + 0.006*"morrison" + 0.006*"royal" + 0.006*"commission" + 0.006*"mother" + 0.006*"sex" + 0.006*"peter"
Topic: 2 Word: 0.020*"man" + 0.015*"police" + 0.013*"charge" + 0.011*"crash" + 0.011*"murder" + 0.011*"woman" + 0.008*"court" + 0.008*"car" + 0.008*"death" + 0.008*"shoot"
Topic: 3 Word: 0.008*"friday" + 0.006*"finance" + 0.005*"coal" + 0.005*"april" + 0.004*"cattle" + 0.004*"kohler" + 0.004*"port" + 0.004*"new" + 0.004*"mix" + 0.004*"social"
Topic: 4 Word: 0.015*"drum" + 0.009*"monday" + 0.008*"grandstand" + 0.008*"stories" + 0.007*"michael" + 0.006*"june" + 0.005*"december" + 0.005*"quiz" + 0.005*"outback" + 0.005*"attempt"
Topic: 5 Word: 0.011*"market" + 0.009*"price" + 0.007*"turnbull" + 0.006*"wall" + 0.006*"weather" + 0.006*"austra

In [66]:
processed_docs[43]

['die', 'korean', 'subway', 'passengers', 'phone', 'help']

In [67]:

for index, score in sorted(lda_model[bow_corpus[43]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.29999998211860657	 
Topic: 0.025*"election" + 0.022*"queensland" + 0.013*"tasmania" + 0.012*"nsw" + 0.012*"speak" + 0.012*"work" + 0.011*"country" + 0.011*"power" + 0.010*"end" + 0.010*"drum"

Score: 0.15714289247989655	 
Topic: 0.024*"win" + 0.020*"market" + 0.018*"die" + 0.017*"world" + 0.014*"cup" + 0.012*"bank" + 0.011*"take" + 0.009*"train" + 0.009*"beat" + 0.008*"scott"

Score: 0.15714287757873535	 
Topic: 0.043*"australia" + 0.020*"day" + 0.018*"south" + 0.017*"china" + 0.016*"north" + 0.015*"test" + 0.010*"west" + 0.009*"war" + 0.008*"australias" + 0.008*"rescue"

Score: 0.15714284777641296	 
Topic: 0.037*"trump" + 0.012*"rural" + 0.012*"plan" + 0.012*"royal" + 0.010*"council" + 0.010*"deal" + 0.010*"commission" + 0.010*"farmers" + 0.009*"price" + 0.009*"business"

Score: 0.1571388989686966	 
Topic: 0.018*"health" + 0.016*"federal" + 0.012*"gold" + 0.012*"help" + 0.012*"coast" + 0.009*"public" + 0.009*"say" + 0.009*"michael" + 0.009*"violence" + 0.009*"png"

Score: 0.

In [68]:
for index, score in sorted(lda_model_tfidf[bow_corpus[43]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.4109349548816681	 
Topic: 0.025*"election" + 0.022*"queensland" + 0.013*"tasmania" + 0.012*"nsw" + 0.012*"speak" + 0.012*"work" + 0.011*"country" + 0.011*"power" + 0.010*"end" + 0.010*"drum"

Score: 0.19113187491893768	 
Topic: 0.037*"trump" + 0.012*"rural" + 0.012*"plan" + 0.012*"royal" + 0.010*"council" + 0.010*"deal" + 0.010*"commission" + 0.010*"farmers" + 0.009*"price" + 0.009*"business"

Score: 0.19041310250759125	 
Topic: 0.024*"win" + 0.020*"market" + 0.018*"die" + 0.017*"world" + 0.014*"cup" + 0.012*"bank" + 0.011*"take" + 0.009*"train" + 0.009*"beat" + 0.008*"scott"

Score: 0.12179766595363617	 
Topic: 0.023*"government" + 0.013*"new" + 0.013*"school" + 0.012*"say" + 0.012*"abc" + 0.011*"cut" + 0.010*"job" + 0.009*"minister" + 0.008*"morrison" + 0.008*"turnbull"

Score: 0.01428811065852642	 
Topic: 0.037*"police" + 0.027*"court" + 0.022*"charge" + 0.018*"murder" + 0.018*"face" + 0.016*"man" + 0.014*"accuse" + 0.012*"high" + 0.012*"death" + 0.012*"trial"

Score: 0.01

In [69]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.34999018907546997	 Topic: 0.037*"trump" + 0.012*"rural" + 0.012*"plan" + 0.012*"royal" + 0.010*"council"
Score: 0.1833333522081375	 Topic: 0.025*"man" + 0.018*"crash" + 0.016*"news" + 0.015*"live" + 0.015*"kill"
Score: 0.18333333730697632	 Topic: 0.016*"miss" + 0.010*"flood" + 0.010*"age" + 0.010*"search" + 0.010*"nrl"
Score: 0.18333324790000916	 Topic: 0.025*"election" + 0.022*"queensland" + 0.013*"tasmania" + 0.012*"nsw" + 0.012*"speak"
Score: 0.016676558181643486	 Topic: 0.018*"health" + 0.016*"federal" + 0.012*"gold" + 0.012*"help" + 0.012*"coast"
Score: 0.01666666939854622	 Topic: 0.023*"government" + 0.013*"new" + 0.013*"school" + 0.012*"say" + 0.012*"abc"
Score: 0.01666666753590107	 Topic: 0.024*"win" + 0.020*"market" + 0.018*"die" + 0.017*"world" + 0.014*"cup"
Score: 0.01666666753590107	 Topic: 0.043*"australia" + 0.020*"day" + 0.018*"south" + 0.017*"china" + 0.016*"north"
Score: 0.01666666753590107	 Topic: 0.048*"australian" + 0.024*"donald" + 0.015*"interview" + 0.01